In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import pandas as pd
from matplotlib import pyplot as plt

from utils import load_img

In [2]:
#try to load train dataset
train_path = "./dataset/train/"
val_path = "./dataset/val/"

train_set = pd.read_csv(train_path + "train_with_answers.csv")

img = load_img(train_set.iloc[4]["file_name"], "train")

print(img)

#plot image
train_set.head()

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1533x1659 at 0x1DDADEE3DD0>


,file_name,question,option1,option2,option3,option4,answer
0,6c2974c9-59f8-4d07-914a-cc3d3d9c4c2f.jpg,2020年の研究開発費が最も高い企業はどれですか？,アメリカの企業,ヨーロッパの企業,日本の企業,Johnson & Johnson,4
1,75d033f9-fc2f-4dc8-9fcd-30f93383a2cd.jpg,ユーザー属性調査において、どの項目が『どちらともいえない』の割合が最も高いですか？,美容に関心がある,ファッションに関心がある,アートに関心がある,スポーツに関心がある,4
2,13b3502d-ca3e-4581-b276-382dd35d8a0b.jpg,この雑誌の発行月は何月ですか？,1月,7月,3月,5月,3
3,2c18497c-825a-4d79-a013-6385a26e9193.jpg,2019年における韓国からの入国者数が最も多い空港はどれですか？,新千歳,関西,羽田,成田,2
4,1b3b926d-84a0-4ea3-b7bf-c2dc35d3894f.jpg,What is the total height of the object from th...,125 mm,120 mm,175 mm,55 mm,3


In [3]:
model = Qwen2VLForConditionalGeneration.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", device_map="auto", offload_buffers=True)

min_pixels = 256*28*28
max_pixels = 512*28*28 
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [4]:
import prompts

print(prompts.generate_prompt(train_set.iloc[4], template=0))

Question: What is the total height of the object from the base to the top of the cylindrical part?
Answer options:
A. 125 mm
B. 120 mm
C. 175 mm
D. 55 mm
Answer with the option’s letter from the given choices directly.


In [5]:
# Preprocess the inputs
text_prompt = processor.apply_chat_template(prompts.generate_conversation(prompts.generate_prompt(train_set.iloc[4])), add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(text=[text_prompt], images=[img], padding=True, return_tensors="pt")
inputs = inputs.to('cuda')

print(text_prompt)


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>Question: What is the total height of the object from the base to the top of the cylindrical part?
Answer options:
A. 125 mm
B. 120 mm
C. 175 mm
D. 55 mm
Answer with the option’s letter from the given choices directly.<|im_end|>
<|im_start|>assistant



In [ ]:
# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=64)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, output_ids)]
output_text = processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(output_text)


In [22]:
from torch import cuda
import gc

cuda.empty_cache()
gc.collect()

cuda.get_device_properties(0).total_memory

4294639616

In [3]:
from models import qwen2vl
import prompts

img = load_img(train_set.iloc[4]["file_name"], "train")

model = qwen2vl()
model.infer(img, prompts.generate_prompt(train_set.iloc[4]))

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.03 GiB. GPU 0 has a total capacity of 4.00 GiB of which 840.64 MiB is free. Of the allocated memory 1.93 GiB is allocated by PyTorch, and 26.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)